In [200]:
from analyse_logs import *

op_df = get_op_df(log_path)
op_df


FileNotFoundError: [Errno 2] No such file or directory: './discv5-test/logs/logs.json'

In [ ]:
op_df['reply_received'].value_counts().plot(kind = 'pie', autopct='%1.0f%%', legend=True, title='Operation returned')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots()
df = op_df[~op_df['time'].isna()]
df
sns.violinplot(x='method',y='time', data=df, ax = axes, cut=0)
#axes.set_title('Day Ahead Market')

In [ ]:
from analyse_logs import *

msg_df = get_msg_df(log_path)
msg_df

In [ ]:
#set the missing opids and op_types to all the messages being part of operations
for opid in set([i for i in op_df['opid']]):
    op_type = op_df.loc[op_df['opid'] == opid, 'method'].values[0]
    for req_id in set([i for i in msg_df[msg_df['opid'] == opid]['req_id']]):
        msg_df.loc[msg_df["req_id"] == req_id, "opid"] = opid
        msg_df.loc[msg_df['req_id'] == req_id, 'op_type'] = op_type

msg_df = msg_df.dropna(subset=['opid'])
msg_df

In [ ]:
from matplotlib.lines import Line2D

colors = ['red', 'green', 'blue', 'yellow']
for op_type, group_op_type in msg_df.groupby('op_type'):
    #if(op_type != 'discv5_registerTopic'):
        #continue
    print(op_type)
    fig, ax = plt.subplots()
    legend_elements = []
    added = set()
    for opid, group_opid in group_op_type.groupby('opid'):
        print("\t", op_type)
        i = 0
        sum = 0
        for msg_type, group_msg_type in group_opid.groupby('msg_type'):
            val = len(group_msg_type)
            ax.bar(opid, val, color=colors[i], bottom = sum)
            sum += val
            print("\t\t", msg_type, len(group_msg_type))
            if(msg_type not in added):
                added.add(msg_type)
                legend_elements.append(Line2D([0], [0], color=colors[i], lw=4, label=msg_type))
            i += 1
    ax.legend(handles=legend_elements)
    ax.set_title(op_type)
plt.show()

In [ ]:
msg_df['in_out'].value_counts().plot(kind='pie', autopct='%1.0f%%', legend=True, title='Msgs sent/received')

In [ ]:
import matplotlib.pyplot as plt

sent = msg_df[msg_df['in_out'] == 'out']['node_id'].value_counts().to_dict()
sent = {int(k):int(v) for k,v in sent.items()} #convert IDs to int
received = msg_df[msg_df['in_out'] == 'in']['node_id'].value_counts().to_dict()
received = {int(k):int(v) for k,v in received.items()} #convert IDs to int

width =0.3
plt.bar(sent.keys(), sent.values(), width=width, label = 'sent')
plt.bar([x + width for x in received.keys()], received.values(), width=width, label = 'received')
plt.legend()
plt.title('Messages exchanged')
plt.xlabel('Node ID')
plt.ylabel('#Messages')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots()
df_in = msg_df[msg_df['in_out']=='in']['node_id'].value_counts().rename_axis('node_id').reset_index(name='count')
df_in['in_out'] = 'in'
df_out = msg_df[msg_df['in_out']=='out']['node_id'].value_counts().rename_axis('node_id').reset_index(name='count')
df_out['in_out'] = 'out'

df = pd.concat([df_in, df_out], axis=0)
sns.violinplot(x='in_out', y='count', data=df, ax = axes, cut=0)

In [ ]:
msg_df['msg_type'].value_counts().plot(kind='bar')

In [ ]:
import numpy as np
import math
import hashlib

df2 = msg_df.dropna()
regs = df2[df2['msg_type'] == 'REGTOPIC/v5']['key'].value_counts().to_dict()
#regs = {int(k):int(v) for k,v in sent.items()} #convert IDs to int
queries = df2[df2['msg_type'] == 'TOPICQUERY/v5']['key'].value_counts().to_dict()
#received = {int(k):int(v) for k,v in received.items()} #convert IDs to int

width =0.3
plt.bar(regs.keys(), regs.values(), width=width, label = 'Registrations')
plt.bar([x + width for x in queries.keys()], queries.values(), width=width, label = 'Queries')
plt.legend()
plt.title('Messages exchanged')
plt.xlabel('Topic')
plt.ylabel('#Messages')



